In [2]:
import pandas as pd
from tqdm import tqdm_notebook as tqdm

def add_space(x):
    if (' ' not in x)&(len(x)<=5):
        return ' ' + x + ' '
    else:
        return x

def product_name_postprocess(x):
    x = x.replace('-',' ')
    x = x.strip()
    x = add_space(x)
    return x

# 讀取台塑網提供之(寶典)
df1 = pd.read_excel('../data/寶典/台塑企業_ 產品寶典20210303.xlsx',engine='openpyxl')[['公司代號','公司事業部門','品名']]
df2 = pd.read_excel('../data/寶典/寶典.v3.台塑網.20210901.xlsx',engine='openpyxl')[['CODIV','DIVNM','ITEMNM']].rename(columns={'ITEMNM':'品名','DIVNM':'公司事業部門','CODIV':'公司代號'})
df3 = pd.read_excel('../data/寶典/寶典.v4.20211001.xlsx',engine='openpyxl').rename(columns={'ITEMNM':'品名','DIVNM':'公司事業部門','CODIV':'公司代號'})
df4 = pd.read_excel('../data/寶典/寶典.v5.20211006.xlsx',engine='openpyxl').rename(columns={'ITEMNM':'品名','DIVNM':'公司事業部門','CODIV':'公司代號'})
df = df1.append(df2).append(df3).append(df4) # 合併所有寶典
df['品名'] = df['品名'].apply(lambda x:product_name_postprocess(x)) #品名後處理
NAME = 'GLASS FABRICS'

In [3]:
display(df[df['品名']==NAME])
display(df[df['品名']==NAME]['公司代號'].value_counts())

,公司代號,公司事業部門,品名,RIGID,EXTN,memo
609,2G,南亞電子部三處,GLASS FABRICS,NaN,NaN,NaN
983,28,電子材料事業部,GLASS FABRICS,NaN,NaN,NaN
253,28,電子材料事業部,GLASS FABRICS,NaN,NaN,NaN
461,2G,南亞電子部三處,GLASS FABRICS,NaN,NaN,NaN
253,28,電子材料事業部,GLASS FABRICS,NaN,NaN,NaN
461,2G,南亞電子部三處,GLASS FABRICS,郝璟憓,5831,NaN
255,28,電子材料事業部,GLASS FABRICS,NaN,NaN,NaN
465,2G,南亞電子部三處,GLASS FABRICS,郝璟憓,5831,NaN


2G    4
28    3
28    1
Name: 公司代號, dtype: int64

In [4]:
品名2部門 = {}
for i in tqdm(df['品名'].values):
    品名2部門[i] = df.loc[df['品名']==i,'公司事業部門'].values.tolist()
品名2代號 = {}
for i in tqdm(df['品名'].values):
    品名2代號[i] = df.loc[df['品名']==i,'公司代號'].values.tolist()

<ipython-input-4-faf643c1cda1>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(df['品名'].values):


<ipython-input-4-faf643c1cda1>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(df['品名'].values):


In [5]:
display(品名2部門[NAME])
display(品名2代號[NAME])

['南亞電子部三處',
 '電子材料事業部',
 '電子材料事業部',
 '南亞電子部三處',
 '電子材料事業部',
 '南亞電子部三處',
 '電子材料事業部',
 '南亞電子部三處']

['2G', 28, '28', '2G', '28', '2G', '28', '2G']

In [6]:
# 從traindata裡找
train_df = pd.read_csv('../data/preprocess_for_SQUAD_產品.csv')
train_df['Y_label'] = train_df['Y_label'].apply(lambda x:x.strip()) #針對SPEC去除左右空白
display(train_df[(train_df['Y_label']==NAME)])
品名2代號訓練資料 = {}
for i in tqdm(train_df['Y_label'].values):
    品名2代號訓練資料[i] = train_df.dropna(subset=['EXPNO'],axis=0).loc[train_df['Y_label']==i,'EXPNO'].unique().tolist()
# 根據品名從訓練資料搜索EXPNO,然後把EXPNO代入保典裡找公司部門
def find_department(x):
    try:
        return [df.loc[df['公司代號']==品名2代號訓練資料[x],'公司事業部門'].value_counts().sort_values().index[-1]]
    except:
        return ['not from_pretrained']
display(品名2代號訓練資料[NAME])
display(find_department(NAME))

,Unnamed: 0,string_X_train,Y_label,EXPNO,from,string_Y_1,string_Y_2
2529,3141,GLASS FABRICS CIF JAPANESE MAIN PORT,GLASS FABRICS,2G,202102.xlsx,1,14
2530,3142,GLASS FABRICS CIF JAPANESE MAIN PORT,GLASS FABRICS,2G,202102.xlsx,1,14
2531,3143,GLASS FABRICS CIF JAPANESE MAIN PORT,GLASS FABRICS,2G,202102.xlsx,1,14
2532,3144,GLASS FABRICS CIF JAPANESE MAIN PORT,GLASS FABRICS,2G,202102.xlsx,1,14
2533,3145,GLASS FABRICS CIF JAPANESE MAIN PORT,GLASS FABRICS,2G,202102.xlsx,1,14
...,...,...,...,...,...,...,...
11299,20375,DESCRIPTION GLASS FABRICS QUANTITY560000MAMOUN...,GLASS FABRICS,2G,202012161231.xlsx,12,25
11300,20376,DESCRIPTION GLASS FABRICS QUANTITY560000MAMOUN...,GLASS FABRICS,2G,202012161231.xlsx,12,25
11301,20377,DESCRIPTION GLASS FABRICS QUANTITY560000MAMOUN...,GLASS FABRICS,2G,202012161231.xlsx,12,25
11302,20378,DESCRIPTION GLASS FABRICS QUANTITY560000MAMOUN...,GLASS FABRICS,2G,202012161231.xlsx,12,25


<ipython-input-6-120c27a2064d>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(train_df['Y_label'].values):


['2G']

['not from_pretrained']

In [7]:
import numpy as np

np.save('../data/對應表/品名2部門.npy',品名2部門)
np.save('../data/對應表/品名2代號.npy',品名2代號)
np.save('../data/對應表/品名2代號訓練資料.npy',品名2代號訓練資料)